# Jigsaw Unintended Bias in Toxicity Classification


## Libraries

In [1]:
!pip install contractions

In [54]:
import pandas as pd
from prettytable import PrettyTable
import nltk
from nltk.corpus import stopwords
from tqdm import trange
import contractions
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from tqdm import tqdm
from copy import copy
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence


In [3]:
nltk.download('stopwords')
stopwords_list = stopwords.words("english")
pd.set_option("display.max_colwidth", None)

In [4]:
EPOCH = 50
EMBED_SIZE = 300

## Helper Function(s)

In [5]:
def simplify_comment(comment):
    simplified_comment = contractions.fix(comment)
    simplified_comment = simplified_comment.replace("\\n", ' ')
    simplified_comment = simplified_comment.replace("\\r", ' ')
    simplified_comment = simplified_comment.replace("\\", ' ')
    simplified_comment = re.sub("[^A-Za-z0-9]+", ' ', simplified_comment)
    
    # in order to prevent unwanted blanks and
    # if a word is one of the stop words.
    return ' '.join(word.strip().lower() for word in simplified_comment.split() if not word in stopwords_list)

## Data Cleansing

In [6]:
df = pd.read_csv("../input/jigsaw-unintended-bias-in-toxicity-classification/train.csv")
df = df.sample(len(df)//10)
df_test = pd.read_csv("../input/jigsaw-unintended-bias-in-toxicity-classification/test.csv")
df.head(10)

In [7]:
pt = PrettyTable()
pt.field_names = ["Area", "Count"]
pt.add_row(["Features", len(df.columns)])
pt.add_row(["Train data rows", len(df)])
pt.add_row(["Test data rows", len(df_test)])
print(pt)

In [8]:
most_toxic = df.sort_values(by=["target"]).iloc[-1, :]
least_toxic = df.sort_values(by=["target"]).iloc[0, :]

In [9]:
print(f"Most toxic comment: {most_toxic['comment_text']}")

In [10]:
print(f"Least toxic comment: {least_toxic['comment_text']}")

In [11]:
df["comment_text"] = df["comment_text"].apply(lambda row: simplify_comment(row))

In [12]:
df["comment_text"].head()

In [13]:
df_test["comment_text"] = df_test["comment_text"].apply(lambda row: simplify_comment(row))

In [14]:
df_test["comment_text"].head()

In [15]:
df.loc[df["target"] >= 0.5, "target"] = 1
df.loc[df["target"] < 0.5, "target"] = 0

In [16]:
df.isnull().sum()/len(df)

In [17]:
all_words = []
maxlen = -9999
for sentence in tqdm(df["comment_text"]):
    maxlen = maxlen if maxlen > len(sentence.split()) else len(sentence.split())
    for word in sentence.split():
        all_words.append(word)
all_words = set(all_words)
num_words = len(all_words)//10

## Data Visualization

In [18]:
sns.displot(df["target"], bins=np.arange(0, 1, 0.1), stat="density")

In [19]:
sns.displot(df, x="target", kind="kde")

In [45]:
text = str(df.loc[df["target"]==1.0]["comment_text"].sample(len(df)//100))
wc = WordCloud(
    background_color = 'white', 
    width = 1920, 
    height = 1080,
    )
wc.generate_from_text(text)
plt.figure(figsize = (12, 12))
plt.title("Target > 0.7")
plt.axis("off") 
plt.tight_layout(pad = 0) 
plt.imshow(wc) 
plt.show()

In [46]:
text = str(df.loc[df["target"]==0.0]["comment_text"].sample(len(df)//10))
wc = WordCloud(
    background_color = 'white', 
    width = 1920, 
    height = 1080,
    )
wc.generate_from_text(text)
plt.figure(figsize = (12, 12))
plt.title("Target < 0.7")
plt.axis("off") 
plt.tight_layout(pad = 0) 
plt.imshow(wc) 
plt.show()

## Data Preprocessing

In [22]:
df_test["comment_text"].head()

In [23]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(
    df["comment_text"].values.tolist() + df_test["comment_text"].values.tolist())

In [24]:
X_train = tokenizer.texts_to_sequences(df["comment_text"])
X_test = tokenizer.texts_to_sequences(df_test["comment_text"])

In [25]:
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen=maxlen)

## Model Building

In [26]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip
!unzip crawl*.zip

In [27]:
embeddings_index = {}
with open("crawl-300d-2M.vec", encoding="utf8") as f:
  for line in f.readlines():
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype="float32")
    embeddings_index[word] = coefs

print(len(tokenizer.word_index))

embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, EMBED_SIZE))
num_words_in_embedding = 0
for word, i in tokenizer.word_index.items():
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    num_words_in_embedding += 1
    embedding_matrix[i] = embedding_vector

print(embedding_matrix.shape)

In [28]:
input_text_lstm = tf.keras.layers.Input(shape=(maxlen,), dtype='float32')

x = tf.keras.layers.Embedding(len(tokenizer.word_index) + 1,
                                    300,
                                    weights=[embedding_matrix],
                                    input_length=maxlen,
                                    trainable=False)(input_text_lstm)
x = tf.keras.layers.SpatialDropout1D(0.2)(x)
x = tf.keras.layers.LSTM(512, return_sequences=True, dropout=0.5)(x)
x = tf.keras.layers.LSTM(256, return_sequences=True, dropout=0.5)(x)
x = tf.keras.layers.LSTM(128, return_sequences=True, dropout=0.5)(x)
x = tf.keras.layers.Conv1D(256, kernel_size=3, padding="valid", activation="relu")(x)

x = tf.keras.layers.concatenate(
    [
        tf.keras.layers.GlobalAveragePooling1D()(x),
        tf.keras.layers.GlobalMaxPooling1D()(x)
    ]
)

x = tf.keras.layers.Dense(512, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)

x = tf.keras.layers.Dense(256, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)

output = tf.keras.layers.Dense(2, activation="softmax")(x)
model = tf.keras.models.Model(inputs=input_text_lstm, outputs=output)

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["acc"])
model.summary()

es = tf.keras.callbacks.EarlyStopping(monitor="val_loss",
                   min_delta=0,
                   patience=3,
                   verbose=0, mode="auto")

best_model = "./model_{epoch:02d}.h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(best_model, monitor="val_loss", verbose=0, save_best_only=False, mode='auto')
# Fit the model
model.fit(X_train, pd.get_dummies(df["target"]), batch_size=256, epochs=EPOCH, callbacks=[es, checkpoint], validation_split=0.1)


## Testing the Model

In [52]:
LABELS = {
    0: "Non-Toxic",
    1: "Toxic"
}

In [60]:
test_sentences = [
    "Hello, I'm in love with your show. It is terrific! Friends is the best show ever..",
    "No way this series is gonna be watched by millions of people all around the globe. It is a piece of garbage. Even though I like the lightning, both cast and the director are the worst in industry.",
    "This video sucks. The quality of it isn't good. I personally wouldn't even watch it if my girlfriend hasn't been recommended it to me.",
    "Totally disagree with you. Fuck it. Enough with you stupidity!"
]

test_pt = PrettyTable()
test_pt.field_names = ["Test Sentence", "Predicted Result"]
test_pt.align["Test Sentence"] = "l"

for sentence in test_sentences:
    raw_sentence = copy(sentence)
    sentence = simplify_comment(sentence)
    sentence = tokenizer.texts_to_sequences([sentence])
    sentence = tf.keras.preprocessing.sequence.pad_sequences(sentence, maxlen=maxlen)
    result = model.predict(sentence)[0]
    argmax = np.argmax(result)
    # print(f"The sentence is %{result[argmax]*100:.2f} {LABELS[argmax]}")
    test_pt.add_row([raw_sentence, f"%{result[argmax]*100:.2f} {LABELS[argmax]}"])
    
print(test_pt)